In [3]:
from datasets import load_dataset
import random
random.seed(42)

sample_size_train = 133166
sample_size_test = 1268
dataset = load_dataset("Angelectronic/IWSLT15_English_Vietnamese")
random_indices_train = random.sample(range(len(dataset['train'])), sample_size_train)
dataset['train'] = dataset['train'].select(random_indices_train)

random_indices_test = random.sample(range(len(dataset['test'])), sample_size_test)
dataset['test'] = dataset['test'].select(random_indices_test)

dataset

Found cached dataset parquet (C:/Users/lg/.cache/huggingface/datasets/Angelectronic___parquet/Angelectronic--IWSLT15_English_Vietnamese-d3d4a119d05830c0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 133166
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 1268
    })
})

In [ ]:
from gradio_client import Client

client = Client("https://skier8402-mistral-super-fast.hf.space/")

def domain_specific(text):
    global client
    prompt_template = '''Does this sentence contain domain-specific words? Return 1 if it does, 0 otherwise. Don't elaborate any further. Sentence: "{}"'''

    result = client.predict(
				prompt_template.format(text),
				0.5,	# int | float (numeric value between 0.0 and 1.0) in 'Temperature' Slider component
				128,	# int | float (numeric value between 0 and 1048) in 'Max new tokens' Slider component
				0.9,	# int | float (numeric value between 0.0 and 1) in 'Top-p (nucleus sampling)' Slider component
				1.2,	# int | float (numeric value between 1.0 and 2.0) in 'Repetition penalty' Slider component
				api_name="/chat"
    )

    return result.strip()

In [ ]:
import json
from tqdm import tqdm

def save_data_to_disk(data):
    with open('iwslt15_en_vi_domain_specific.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

data_train = []
with open('iwslt/iwslt15_en_vi_domain_specific.json', 'r', encoding='utf-8') as f:
    data_train = json.load(f)        

save_step = 200
for i in tqdm(range(len(dataset['train']))):
    data = dict(dataset['train'][i])

    if i % save_step == 0:
        save_data_to_disk(data_train)
        print(f"Processing {i}th example")

    if data['id'] in [d['id'] for d in data_train]:
        continue
    
    try:
        resp = domain_specific(dataset['train'][i]['translation']['en'])
        if resp[0] == '1' or resp[:3] == 'Yes':
            data['domain_specific'] = True
        elif resp[0] == '0' or resp[:2] == 'No':
            data['domain_specific'] = False
        else:
            data['domain_specific'] = None
            data['domain_specific_response'] = resp
    except:
        save_data_to_disk(data_train)
        client = Client("https://skier8402-mistral-super-fast.hf.space/")

        resp = domain_specific(dataset['train'][i]['translation']['en'])
        if resp[0] == '1' or resp[:3] == 'Yes':
            data['domain_specific'] = True
        elif resp[0] == '0' or resp[:2] == 'No':
            data['domain_specific'] = False
        else:
            data['domain_specific'] = None
            data['domain_specific_response'] = resp
    data_train.append(data)

save_data_to_disk(data_train)

In [4]:
import json 
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

with open('iwslt/iwslt15_en_vi_domain_specific_1.json', 'r', encoding='utf-8') as f:
    data_1 = json.load(f)

with open('iwslt/iwslt15_en_vi_domain_specific_2.json', 'r', encoding='utf-8') as f:
    data_2 = json.load(f)

with open('iwslt/iwslt15_en_vi_domain_specific_3.json', 'r', encoding='utf-8') as f:
    data_3 = json.load(f)

data_123 = data_1 + data_2 + data_3
print(len(data_123), len(data_1), len(data_2), len(data_3))
 
id_list = [d['id'] for d in data_123]
true_ids = [d['id'] for d in data_123 if d['domain_specific'] == True]
null_ids_true = [d['id'] for d in data_123 if d['domain_specific'] == None and 'translation' in d]
null_ids_false = [d['id'] for d in data_123 if d['domain_specific'] == None and 'translation' not in d]

test_dm_dict = dict()
test_dm_dict['id'] = []
test_dm_dict['translation'] = []
for d in dataset['train']:
    if d['id'] in true_ids:
        test_dm_dict['id'].append(d['id'])
        test_dm_dict['translation'].append(d['translation'])

def delete_example(example):
    if example['id'] in null_ids_false or example['id'] in true_ids:
        return False
    return True

def change_translation(example):
    if example['id'] in null_ids_true:
        index = id_list.index(example['id'])
        example['translation'] = data_123[index]['translation']
    return example

dataset['train'] = dataset['train'].filter(delete_example).map(change_translation)

new_dataset = DatasetDict({'train': dataset['train'], 'test': dataset['test'], 'domain_specific_test': Dataset.from_dict(test_dm_dict)})
print(new_dataset)

4718 1572 1573 1573


Loading cached processed dataset at C:\Users\lg\.cache\huggingface\datasets\Angelectronic___parquet\Angelectronic--IWSLT15_English_Vietnamese-d3d4a119d05830c0\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-8198e0d7d9067204.arrow


Map:   0%|          | 0/131621 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 131621
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 1268
    })
    domain_specific_test: Dataset({
        features: ['id', 'translation'],
        num_rows: 1542
    })
})


In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": device}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

embedding = hf.embed_query("hi this is harrison")
len(embedding)

384

In [6]:
from langchain.vectorstores import FAISS
import json
from langchain.docstore.document import Document

tm_docs = [Document(page_content=new_dataset['train'][i]['translation']['en'], metadata={"id": new_dataset['train'][i]['id']}) for i in range(len(new_dataset['train']))]
db = FAISS.from_documents(tm_docs, hf)
db.save_local("tm_vectorstore")
db.similarity_search("Nobody 's ever done it before , so I 'm going to go do it .")

[Document(page_content="Nobody 's ever done it before , so I 'm going to go do it .", metadata={'id': 29184}),
 Document(page_content="I just did something I 've never done before .", metadata={'id': 43182}),
 Document(page_content='I did something that nobody else had done .', metadata={'id': 109867}),
 Document(page_content="So , talk about risk taking . I 'm going to do somebody that nobody likes .", metadata={'id': 51443})]

In [7]:
from langchain.vectorstores import FAISS

vector_db = FAISS.load_local("tm_vectorstore", hf)
k = 3
retriver = vector_db.as_retriever(search_kwargs={'k': k})
relevant_docs = retriver.get_relevant_documents("what did he say about ketanji brown jackson")
relevant_docs

[Document(page_content='He was interviewed once , and he said the following .', metadata={'id': 45660}),
 Document(page_content='Asked him what this said .', metadata={'id': 59394}),
 Document(page_content="Here 's what he had to say .", metadata={'id': 84341})]

In [8]:
import copy

dataset_copy = copy.deepcopy(new_dataset)

id_list = [data['id'] for data in new_dataset['train']]
rand_id = random.sample(id_list, len(new_dataset['train'])//2)

def get_relevant_docs_train(example):    
    if example['id'] in rand_id:
        relevant_docs = retriver.get_relevant_documents(example['translation']['en'])
        relevant_docs = [doc for doc in relevant_docs if doc.metadata['id'] != example['id']]
        if len(relevant_docs) > k - 1:
            relevant_docs.pop()

        instructs = ""              
        for doc in relevant_docs:
            index = id_list.index(int(doc.metadata['id']))
            tm_doc = new_dataset['train'][index]

            instructs += f"<English> : {tm_doc['translation']['en']}\n<Vietnamese> : {tm_doc['translation']['vi']}\n\n"
            
        example['translation']['en'] = instructs + "<English> : " + example['translation']['en'] + "\n" + "<Vietnamese> : "
    else:
        example['translation']['en'] = "<English> : " + example['translation']['en'] + "\n" + "<Vietnamese> : "
    
    return example

def get_relevant_docs_test(example):    
    relevant_docs = retriver.get_relevant_documents(example['translation']['en'])
    relevant_docs = [doc for doc in relevant_docs if doc.metadata['id'] != example['id']]
    if len(relevant_docs) > k - 1:
        relevant_docs.pop()

    instructs = ""              
    for doc in relevant_docs:
        index = id_list.index(int(doc.metadata['id']))
        tm_doc = new_dataset['train'][index]

        instructs += f"<English> : {tm_doc['translation']['en']}\n<Vietnamese> : {tm_doc['translation']['vi']}\n\n"
        
    example['translation']['en'] = instructs + "<English> : " + example['translation']['en'] + "\n" + "<Vietnamese> : "
    return example

dataset_copy['train'] = dataset_copy['train'].map(get_relevant_docs_train)
dataset_copy['test'] = dataset_copy['test'].map(get_relevant_docs_test)
dataset_copy['domain_specific_test'] = dataset_copy['domain_specific_test'].map(get_relevant_docs_test)
# dataset_copy.push_to_hub("Angelectronic/fuzzy_iwslt15_domain_specific")

print(dataset_copy['train'][0]['translation']['en'])

Map:   0%|          | 0/131621 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1542 [00:00<?, ? examples/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Pushing split domain_specific_test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

<English> : I know how to be Mowgli .
<Vietnamese> : 
